In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import os
import glob
import shutil
import zipfile
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [23]:
config = {}
Test = False

if Test:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/'
    config['start'] = 7000
    config['end'] = 8200
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path][config['start']:config['end']]
    print(len(filenames))
else:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/test_soundscapes/'
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path]
    print(len(filenames))

0


In [24]:
from tqdm import tqdm

chunk_feature_dict = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    cur_file = file.replace('.ogg','')
    samples_per_segment = sample_rate * 5
    if len(audio) > 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)
        
    for i in range(0, total_samples+160000, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            chunk = cur_file+'_'+str(int((i/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
            chunk_feature_dict[chunk] = np.ascontiguousarray(flattened_features)

0it [00:00, ?it/s]


In [25]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
bird_cols = list(meta_data['primary_label'].unique().sort())

submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit['row_id'] = 'samples'

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [26]:
rbf_kernel_svm_model = load('/kaggle/input/rbf-kernel-svc-model/rbf_kernel_svm_model.joblib')

In [27]:
for file_name, feature_set in tqdm(chunk_feature_dict.items()):
    df = pd.DataFrame(rbf_kernel_svm_model.predict_proba(feature_set), columns=bird_cols)
    df.insert(loc=0, column='row_id', value=file_name)
    submit = pd.concat([submit,df]).reset_index(drop=True)
    
i = submit[(submit.row_id == 'samples')].index
submit = submit.drop(i).reset_index(drop=True)

0it [00:00, ?it/s]


In [28]:
poor_scorers = ['ashdro1',
 'ashpri1',
 'ashwoo2',
 'aspswi1',
 'barfly1',
 'barswa',
 'bkcbul1',
 'bkrfla1',
 'bkwsti',
 'blaeag1',
 'blhori1',
 'blrwar1',
 'brasta1',
 'brcful1',
 'brfowl1',
 'brnshr',
 'brwjac1',
 'btbeat1',
 'chbeat1',
 'cohcuc1',
 'comkin1',
 'commyn',
 'comros',
 'comtai1',
 'cregos1',
 'crseag1',
 'emedov2',
 'eurcoo',
 'gargan',
 'goflea1',
 'grbeat1',
 'grecou1',
 'greegr',
 'grejun2',
 'grnwar1',
 'grywag',
 'gyhcaf1',
 'heswoo1',
 'inbrob1',
 'indrol2',
 'insbab1',
 'isbduc1',
 'jerbus2',
 'kerlau2',
 'lirplo',
 'litegr',
 'maghor2',
 'maltro1',
 'marsan',
 'niwpig1',
 'nutman',
 'pabflo1',
 'piekin1',
 'placuc3',
 'plapri1',
 'pomgrp2',
 'rorpar',
 'rossta2',
 'rufbab3',
 'ruftre2',
 'rufwoo2',
 'rutfly6',
 'spepic1',
 'spodov',
 'spoowl1',
 'tibfly3',
 'vefnut1',
 'wemhar1',
 'whbwat1',
 'whrmun',
 'wynlau1',
 'yebbab1']

In [29]:
len(poor_scorers)

72

In [30]:
submit.head(5)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1


In [31]:
# meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
# bird_cols = list(meta_data['primary_label'].unique().sort())
for col in bird_cols:
    if col in poor_scorers:
        # print('{0} is poor'.format(col))
        submit[col].values[:] = 0.00000

In [32]:
submit.head(5)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1


In [33]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submit.columns)

In [34]:
submit.to_csv('submission.csv', index=False)